## All in all
1. **Hash tables** are the engines behind Python's high-performance dicts
2. "Any running Python program has many dictionaries active at the same time, even if the user's program code doesn't explicitly use a dictionary" -- A.M.Kuchling

## Generic Mapping Types
1. [collections.abc](https://docs.python.org/3/library/collections.abc.html) -- abstract base classes, which can be used to test whether a class provides a particular interface; for example, whether it is hashable or whether it is a mapping
2. definition of **hashable**
    - An object is hashable if it has a hash value which neer changes during its lifetime (it needs a __hash__() method), and can be compared to other objects (__eq()__ method). Hashable objects which compare equal must hve the smae hash value.
    - what are hashable:
        - atomic immutable types (str, bytes, numeric types)
        - frozen sets
        - tuple (if andonly if all its items are hashable)
    - what are unhashable:
        mutable objects, e.g. list, dictionaries, collections.deque, sets, whose hash value is changable by their mutability

## dict Comprehensions
1. various means to build a dict
    ```python3
    a = dict(one=1, two=2, three=3)
    b = {'one': 1, 'two': 2, 'three': 3}
    c = dict(zip(['one', 'two', 'three'], [1,2,3]))
    d = dict([('one', 1), ('two', 2), ('three', 3)])
    e = dict({'one': 1, 'two': 2, 'three': 3})
    ```
2. dictcomps
    ```python3
    DAIL_CODES = [(86, 'China'), (91, 'India')] 
    country_code = {country: code for code, country in DAIL_CODES} # {'China': 86, 'India': 91}
    upper_country_code = {code: country.upper() for country, code in country_code.items() if code > 90} #{91: 'INDIA'}
    ```

## Overview of Common Mapping Methods
1. dict
2. defaultdict
3. OrderedDict

### Handling Missing Keys with setdefault
1. dict.get(k, default) is common, but when updating the value found (if it is mutable), using either __getitem__ or get is awkward and inefficient.

2. Compare
    - suboptimal script
    ```python3
    import sys
    import re

    WORD_RE = re.compile('\w+')

    index = {}
    with open(sys.argv[1], encoding='utf-8') as fp:
        for line_no, line in enumerate(fp, 1):
            for match in WORD_RE.finditer(line):
                word = match.group()
                column_no = match.start()+1
                location = (line_no, column_no)
                # this is ugly; coded like this to make a point
                occurrences = index.get(word, [])  # <1>
                occurrences.append(location)       # <2>
                index[word] = occurrences          # <3>
    ```
    - optimal
    ```python3
    import sys
    import re

    WORD_RE = re.compile('\w+')

    index = {}
    with open(sys.argv[1], encoding='utf-8') as fp:
        for line_no, line in enumerate(fp, 1):
            for match in WORD_RE.finditer(line):
                word = match.group()
                column_no = match.start()+1
                location = (line_no, column_no)
                index.setdefault(word, []).append(location)  # <1>
     ```

## Mappings with Flaxible Key Lookup
### defaultdict: Another Take on Missing Keys
1. when instantiating a **defaultdict**, you provide a callable that is used to produce a default value whenever __getitem__ is passed a nonexistent key argument. The callable mentioned is held in an instance attribute called **default_factory**
2. example
 
    ```python3
    import sys
    import re
    import collections

    WORD_RE = re.compile('\w+')

    index = collections.defaultdict(list)     # <1>
    with open(sys.argv[1], encoding='utf-8') as fp:
        for line_no, line in enumerate(fp, 1):
            for match in WORD_RE.finditer(line):
                word = match.group()
                column_no = match.start()+1
                location = (line_no, column_no)
                index[word].append(location)  # <2>
    ```

### The __missing__ Method
1. if you subclass dict and provide a __missing__ method, the standard dict.__getitem___ wil call it whenever a key is not found, instead of raising KeyError.
2. a better way to create a user-defined mapping type is to subclass collections.UserDict instead of dict
3. example: a subclass dictionary that supports both str and int key lookup
    ```python3 
    class StrKeyDict0(dict):  # <1>

    def __missing__(self, key):
        if isinstance(key, str):  # <2>
            raise KeyError(key)
        return self[str(key)]  # <3>

    def get(self, key, default=None):
        try:
            return self[key]  # <4>
        except KeyError:
            return default  # <5>

    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()  # <6>
    ```

## Variations of dict
1. collections.OrderedDict
2. collections.ChainMap
3. collections.Counter
4. collections.UserDict
    - A pure Python implementation of a mapping that works like a standard dict. Designed to be subclassed.

## Subclassing UserDict
1. The main reason why it's preferable to subclass from UserDict rather than from dict is the built-in has some mplementation shortcuts that end up forcing us to override methods taht we can just inherit from UserDict with no problems.
2. **UserDict** does not inherit from **dict**, but has an internal dict instance called **data**, which holds the actual items. 
3. UserDict subclasses **MutableMapping**
4. example: StrKeyDict always converts non-string keys to str-on insertion, update and lookup
    ```python3
    import collections

    class StrKeyDict(collections.UserDict):  # <1>

        def __missing__(self, key):  # <2>
            if isinstance(key, str):
                raise KeyError(key)
            return self[str(key)]

        def __contains__(self, key):
            return str(key) in self.data  # <3>

        def __setitem__(self, key, item):
            self.data[str(key)] = item   # <4>
    ```

## Immutable Mappings
1. The mapping types provided by the standard library are all mutable, but sometimes we may need to guarantee taht a user cannot change a mapping by mistake.
2. Since Python3.3, the **types** module provides a wrapper class called **MappingProxyType** which, given a mapping, returns a **mappingproxy** that is a read-only and dynamic view of the original mapping. This means that update to the original mapping can be seen in the mappingproxy, but changes cannot be made through it.
3. example
    ```python3
    >>> from types import MappingProxyType
    >>> d = {1: 'A'}
    >>> d_proxy = MappingProxyType(d)
    >>> d_proxy
    mappingproxy({1: 'A'})
    >>> d_proxy[1]
    'A'
    >>> d_proxy[2]
    Traceback (most recent call last):
      File "<stdin>", line 1, in <module>
    KeyError: 2
    >>> d[2] = 'B'
    >>> d_proxy
    mappingproxy({1: 'A', 2: 'B'})
    ```

## Set Theory
1. set
2. frozenset (immutable sets)

### Set Comprehensions
```python3
from unicodedata import name
{chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i), '')}
#{'¤', '¶', '°', '¢', '$', 'µ', '¥', '=', '±', '%', '®', '÷', '¬', '>', '§', '#', '£', '©', '×', '+', '<'}
```

## dict and set Under the Hood


## Resources
1. [python glossary](https://docs.python.org/3/glossary.html#term-abstract-base-class)
2. https://github.com/python/cpython/blob/master/Objects/dictobject.c
